In [1]:
import os
import pandas as pd
import json
import numpy as np

dataset_dir = os.path.expanduser('~') + os.sep + 'Kaggle' + os.sep + 'turkeyCompetition' + os.sep + 'dataset' + os.sep
train_filename = 'train.json'
test_filename = 'test.json'

In [2]:
#Look into a couple of samples from TRAIN dataset
train_data = []

with open(dataset_dir + train_filename) as f:
    train_data = json.load(f)
    
for idx, sample in enumerate(train_data):
    #print("Keys available: {}".format(sample.keys()))
    #for key in sample.keys():
        #print("Key: {} -- Value: {}".format(key, sample[key]))
    print('\n')
    if idx == 0:
        break

In [3]:
#Look into a couple of samples from VALIDATION dataset
validation_data = []

with open(dataset_dir + test_filename) as f:
    validation_data = json.load(f)
    
for idx, sample in enumerate(validation_data):
    #print("Keys available: {}".format(sample.keys()))
    #for key in sample.keys():
        #print("Key: {} -- Value: {}".format(key, sample[key]))
    print('\n')
    if idx == 0:
        break

In [4]:
import matplotlib.pyplot as plt

#Check distribution of the sequences of audio encoding
distribution = {}
for i in range(1,11):
    distribution[i]=0


for val in train_data:    
    distribution[len(val['audio_embedding'])]+=1

print(len(train_data))
print(distribution)
plt.bar(distribution.keys(), distribution.values(), width=1.0, color='g')
plt.show()

distribution = {}
for i in range(0,256):
    distribution[i]=0

max_val_encoding = 0
for val in train_data:
    for e in val['audio_embedding']:
        for v in e:
            distribution[v] += 1
            
print(distribution)

plt.bar(distribution.keys(), distribution.values(), width=1.0, color='b')
plt.show()

1195
{1: 0, 2: 5, 3: 1, 4: 2, 5: 3, 6: 4, 7: 1, 8: 3, 9: 56, 10: 1120}


<Figure size 640x480 with 1 Axes>

{0: 126642, 1: 2133, 2: 2257, 3: 2306, 4: 2306, 5: 2386, 6: 2415, 7: 2393, 8: 2656, 9: 2591, 10: 2622, 11: 2724, 12: 2712, 13: 2789, 14: 2832, 15: 2931, 16: 2923, 17: 3044, 18: 3047, 19: 3116, 20: 3097, 21: 3161, 22: 3119, 23: 3209, 24: 3163, 25: 3364, 26: 3246, 27: 3213, 28: 3387, 29: 3271, 30: 3310, 31: 3354, 32: 3395, 33: 3523, 34: 3581, 35: 3632, 36: 3596, 37: 3636, 38: 3798, 39: 3723, 40: 3849, 41: 3831, 42: 3835, 43: 3962, 44: 3998, 45: 3921, 46: 4154, 47: 4151, 48: 4349, 49: 4220, 50: 4428, 51: 4447, 52: 4407, 53: 4487, 54: 4573, 55: 4749, 56: 4569, 57: 4627, 58: 4776, 59: 4874, 60: 4826, 61: 4997, 62: 4855, 63: 5074, 64: 5036, 65: 5095, 66: 5111, 67: 5358, 68: 5267, 69: 5526, 70: 5433, 71: 5492, 72: 5438, 73: 5436, 74: 5538, 75: 5501, 76: 5620, 77: 5782, 78: 5860, 79: 5738, 80: 5932, 81: 5783, 82: 6007, 83: 6180, 84: 6080, 85: 6126, 86: 6145, 87: 6085, 88: 6010, 89: 6150, 90: 6272, 91: 6253, 92: 6402, 93: 6449, 94: 6216, 95: 6320, 96: 6463, 97: 6364, 98: 6417, 99: 6535, 100: 66

<Figure size 640x480 with 1 Axes>

In [5]:
#Split the dataset into training and validation to check how the model is going to perform
from sklearn.model_selection import train_test_split

x_train_data, x_val_data = train_test_split(train_data,test_size=0.1,train_size=None,random_state=34,shuffle=True)

xtrain = [i['audio_embedding'] for i in x_train_data]
ytrain = [k['is_turkey'] for k in x_train_data]

xval = [i['audio_embedding'] for i in x_val_data]
yval = [k['is_turkey'] for k in x_val_data]


#Create padding for the samples that have less than 10 entries. 
#Instead of padding with 0, created a padding that repeats the entries
for idx,x in enumerate(xtrain):
    if len(x) < 10:
        for z,i in enumerate(range(len(x), 10)):
            xtrain[idx].append(xtrain[idx][z])
            
for idx,x in enumerate(xval):
    if len(x) < 10:
        for z,i in enumerate(range(len(x), 10)):
            xval[idx].append(xval[idx][z])

ytrain = np.asarray(ytrain)
yval = np.asarray(yval)

In [6]:
#Check dimensions:
print("xtrain: {} x {} x {}".format(len(xtrain), len(xtrain[0]), len(xtrain[0][0])))
print("xval: {} x {} x {}".format(len(xval), len(xval[0]), len(xval[0][0])))

xtrain: 1075 x 10 x 128
xval: 120 x 10 x 128


In [7]:
#Build the model
import tensorflow as tf
from tensorflow.python import debug as tf_debug

tf.logging.set_verbosity(tf.logging.ERROR)
tf.reset_default_graph()
# Hyperparameters
learning_rate = 0.0001
training_epochs = 20
display_step = 1
batch_size = 50

#Size of each sample
num_samples_per_vid = len(xtrain[0])
num_encodings_per_sample = len(xtrain[0][0])

n_input = num_samples_per_vid * num_encodings_per_sample
n_classes = 1 #Is turkey

n_hidden_layer = 256

logdir = '/Users/diogo/Kaggle/turkeyCompetition/logs'

In [8]:
# tf Graph input
with tf.name_scope("variables_scope"):
    x = tf.placeholder("float", [None, num_samples_per_vid, num_encodings_per_sample], name='input')
    y = tf.placeholder("float", [None, n_classes], name='target')
    x_flat = tf.reshape(x, [-1, n_input], name='input_flattened')
    
    
    with tf.name_scope("weights_scope"):
        # Store layers weight & bias
        weights = {
        'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer]), name='hidden_w'),
        'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]), name='out_w')
        }
        tf.summary.histogram("weight_histogram_hidden", weights['hidden_layer'])
        tf.summary.histogram("weight_histogram_out", weights['out'])

        biases = {
        'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer]), name='hidden_b'),
        'out': tf.Variable(tf.random_normal([n_classes]), name='out_b')
        }



    with tf.name_scope("yReal_scope"):
        # Hidden layer with RELU activation
        layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'], name="layer_1")
        layer_1 = tf.nn.relu(layer_1)
        # Output layer with linear activation
        logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'], name="logits")
        tf.summary.histogram("logits_histogram", logits)


In [9]:
with tf.name_scope("training_scope"):
    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y), name='cost')
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost, name='gradDescent')
    tf.summary.histogram("loss_histogram", cost)
    tf.summary.scalar("loss_scalar", cost)

# Calculate accuracy
with tf.name_scope("accuracy_scope"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    tf.summary.histogram("accurace_scalar", accuracy)
    tf.summary.scalar("accurace_scalar", accuracy)
    

In [10]:
def get_batch(features, targets, batch_size=10):
    batch_x = np.zeros(shape=(batch_size, num_samples_per_vid, num_encodings_per_sample))
    batch_y = np.zeros(shape=(batch_size, n_classes))
    
    counter = 0
    for val_x, val_y in zip(features, targets):
        batch_x[counter] = val_x
        batch_y[counter] = val_y
        counter += 1
        if counter % batch_size == 0:
            yield (np.asarray(batch_x), np.asarray(batch_y))
            counter = 0
            batch_x = np.zeros(shape=(batch_size, num_samples_per_vid, num_encodings_per_sample))
            batch_y = np.zeros(shape=(batch_size, n_classes))
            

In [11]:
# Initializing the variables
init = tf.global_variables_initializer()


# TensorBoard - Write the default graph out so we can view it's structure
merged_summary_op = tf.summary.merge_all()
tbWriter = tf.summary.FileWriter(logdir)

# Launch the graph
with tf.Session() as sess:
    #sess = tf_debug.TensorBoardDebugWrapperSession(sess, "Diogos-MacBook-Pro-2.local:8080")
    sess.run(init)
    tbWriter.add_graph(sess.graph)
    
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        idx = 0
        for batch_x, batch_y in get_batch(xtrain, ytrain, batch_size=batch_size):
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            
            if idx % 5 == 0:
                summary = sess.run(merged_summary_op, feed_dict={x: batch_x, y: batch_y})
                tbWriter.add_summary(summary, idx)
            
        if epoch % 2 == 0:
            yy = yval.reshape(len(yval), n_classes)
            xx = np.asarray(xval)
            valid_accuracy = sess.run(
                accuracy,
                feed_dict={
                    x: xx,
                    y: yy})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(
                epoch,
                valid_accuracy))

Epoch 0   - Validation Accuracy: 1.0
Epoch 2   - Validation Accuracy: 1.0
Epoch 4   - Validation Accuracy: 1.0
Epoch 6   - Validation Accuracy: 1.0


InvalidArgumentError: Infinity in summary histogram for: variables_scope/yReal_scope/logits_histogram
	 [[node variables_scope/yReal_scope/logits_histogram (defined at <ipython-input-8-7df0c3d009d6>:30)  = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](variables_scope/yReal_scope/logits_histogram/tag, variables_scope/yReal_scope/logits)]]

Caused by op 'variables_scope/yReal_scope/logits_histogram', defined at:
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/diogo/Kaggle/turkeyCompetition/venv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/diogo/Kaggle/turkeyCompetition/venv/lib/python3.5/site-pa